`63070501061 S.RAKNA`

# Logistic Regression Gradient Descent

> 2.5 hrs

#### Modify `Logistic Regression Gradient Descent.ipynb` for Iris classification to use the cost function and Jacobian function and the `BFGS` optimization method in `SciPy` library to:

> 20 points

##### a. Report the total classification accuracy (score) for the Test data by finding the highest probability class as the output.

In [1]:
import math

import matplotlib as plt
import numpy as np
from scipy.optimize import minimize
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
class LogisticRegression():
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

        self.theta = (
            20 * np.random.random_sample((X_train.shape[1], 1)) - 10).flatten()
        self.predict_y = None
        self.predict_y_before = None
        self.correct = None

    def stable_sigmoid(self, x) -> np.ndarray:
        return np.where(
            x < 0,
            np.exp(x)/(1 + np.exp(x)),
            1/(1 + np.exp(-x))
        )

    def h(self, theta, X):  # theta is a n x 1 vector
        # X is a mxn matrix
        #  [x0 x1 ... x_n-1]_1 where n-1 is size of feature vector (no. features)
        #  [x0 x1 ... x_n-1]_2 REMEMBER x0 is 1.
        #  ...
        #  [x0 x1 ... x_n-1]_m where m is number of data points
        # for matrix X_mxn, and vector theta_nx1 this will return a vector mx1
        # print("shape X, theta = ", X.shape, theta.shape)
        z = X @ theta  # X is m x n, theta is n x 1. Result z is mx1.
        return self.stable_sigmoid(z)  # returns vector mx1

    def J(self, theta, X, y):
        # X[i] is 1 data point
        # y = X @ theta where X is m x n and theta is n x 1. y is m x 1.
        # y[i] is 0 or 1 only for data belongs to this class or not this class
        # m is number of data points.  n is number of features
        # first column of X contain all 1's
        # Assume we have J = (-1/m)SUM_{i=1..m}[y_i * log(h(theta, X[i])) +
        #                                          (1-y_i)*log(1-h(theta, X[i])) ]
        m = X.shape[0]  # m - number of data points
        # n - no. features + 1, because first column X is all 1 for intercepts.
        n = X.shape[1]
        H = self.h(theta, X)
        # print ("In function E, m, n = ", m, n, "y.shape = ", y.shape, "h shape = ", T1.shape)
        cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m
        return cost  # returns a scalar

    def gradient(self, theta, X, y):
        m = X.shape[0]  # m - number of data points
        # n - no. features + 1, because first column X is all 1 for intercepts.
        n = X.shape[1]
        # this is n x m * m x 1 to get n x 1
        dJ_dt = X.T @ (self.h(theta, X) - y)/m
        return dJ_dt  # an array n x 1

    def minimize(self):
        res = minimize(self.J, self.theta, args=(self.X_train, self.y_train),
                       method='BFGS', jac=self.gradient, options={'disp': True})
        self.theta = res.x

    def predict(self):
        self.predict_y_before = self.h(self.theta, self.X_test)
        self.predict_y = np.where(self.predict_y_before >= 0.5, 1, 0)
        # correct in percentage
        self.correct = np.sum(self.predict_y == self.y_test) / \
            self.y_test.shape[0] * 100

    def print(self):
        print("theta = ", self.theta)
        print("predict_y = ", self.predict_y)
        print("correct = ", self.correct)


In [3]:
X, y = load_iris(return_X_y=True)
print("5 samples Iris data X = \n", X[:5][:])
print("All Iris data y = \n", y)
# Use sklearn's library to split the data into training and testing sets with ratio 75% to 25%.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0)
print("First 5 samples of X_train: \n", X_train[0:5][:])
print("First 5 samples of y_train: \n", y_train[0:5])


5 samples Iris data X = 
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
All Iris data y = 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
First 5 samples of X_train: 
 [[5.9 3.  4.2 1.5]
 [5.8 2.6 4.  1.2]
 [6.8 3.  5.5 2.1]
 [4.7 3.2 1.3 0.2]
 [6.9 3.1 5.1 2.3]]
First 5 samples of y_train: 
 [1 1 2 0 2]


In [4]:
# class
y0_train = np.where(y_train == 0, 1, 0)
y1_train = np.where(y_train == 1, 1, 0)
y2_train = np.where(y_train == 2, 1, 0)
y0_test = np.where(y_test == 0, 1, 0)
y1_test = np.where(y_test == 1, 1, 0)
y2_test = np.where(y_test == 2, 1, 0)

In [5]:
lr0 = LogisticRegression(X_train, y0_train, X_test, y0_test)
lr0.minimize()
lr0.predict()
lr0.print()

Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 22
         Function evaluations: 29
         Gradient evaluations: 29
theta =  [  2.40932953   7.11747435 -13.04678157  -4.97442521]
predict_y =  [0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 1
 0]
correct =  100.0


/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: divide by zero encountered in log
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m
/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: invalid value encountered in matmul
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m


In [6]:
lr1 = LogisticRegression(X_train, y1_train, X_test, y1_test)
lr1.minimize()
lr1.predict()
lr1.print()

Optimization terminated successfully.
         Current function value: 0.510000
         Iterations: 29
         Function evaluations: 37
         Gradient evaluations: 37
theta =  [ 0.6721146  -1.79056047  1.10285494 -3.07298673]
predict_y =  [0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
correct =  63.1578947368421


In [7]:
lr2 = LogisticRegression(X_train, y2_train, X_test, y2_test)
lr2.minimize()
lr2.predict()
lr2.print()

         Current function value: nan
         Iterations: 21
         Function evaluations: 33
         Gradient evaluations: 33
theta =  [-14.64059462 -15.59542351  23.67060079  12.26399725]
predict_y =  [1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0
 1]
correct =  97.36842105263158


/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: divide by zero encountered in log
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m
/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: invalid value encountered in matmul
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m
/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: divide by zero encountered in log
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m
/var/folders/dm/8jvmtg054t9fvkdq5lplxb200000gn/T/ipykernel_13306/2387489400.py:45: RuntimeWarning: invalid value encountered in matmul
  cost = -(y.T @ np.log(H) + (1-y).T @ np.log(1-H))/m


In [8]:
print(f"class0:\n{lr0.predict_y_before}\n")
print(f"class1:\n{lr1.predict_y_before}\n")
print(f"class2:\n{lr2.predict_y_before}\n")

class0:
[4.38212945e-20 1.79300031e-12 1.00000000e+00 1.04034151e-23
 9.99999845e-01 2.48220198e-23 9.99999991e-01 5.26646010e-14
 3.53388749e-15 3.67074732e-11 4.62029378e-21 7.07861471e-13
 6.52363061e-15 1.41754766e-14 4.91500836e-15 9.99999992e-01
 3.19213254e-14 1.85482036e-14 9.99998073e-01 1.00000000e+00
 2.69022569e-18 2.48110698e-14 9.99953705e-01 9.99993740e-01
 1.13839096e-16 1.00000000e+00 9.99996474e-01 1.89956760e-12
 3.03830589e-09 9.99998456e-01 1.68494380e-19 1.53240580e-14
 9.99999953e-01 1.00752777e-16 7.38755586e-22 4.61850928e-12
 9.99999963e-01 1.86269290e-18]

class1:
[0.05386458 0.80719525 0.05243486 0.75591662 0.15608546 0.06066161
 0.08358745 0.38375706 0.63374856 0.37821459 0.78889008 0.25447369
 0.64156515 0.45483053 0.44732381 0.12831299 0.30862175 0.55135593
 0.17895692 0.04112083 0.12007588 0.22192688 0.20085992 0.21317011
 0.25272501 0.05385845 0.07515207 0.4309144  0.45232489 0.10047486
 0.32852188 0.19958415 0.15029609 0.19794796 0.21473457 0.20741724


In [9]:
max_class = []
for i in range(len(lr0.predict_y_before)):
    max_class.append(np.argmax([
        lr0.predict_y_before[i],
        lr1.predict_y_before[i],
        lr2.predict_y_before[i]
    ]))

print(f"max_class:\n{max_class}\n")
print(f"y_test:\n{y_test}\n")
print(f"max_class == y_test:\n{max_class == y_test}\n")
print(f"correct:\n{np.sum(max_class == y_test) / len(y_test) * 100}\n")


max_class:
[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0, 2, 2, 0, 2, 2, 1, 0, 2]

y_test:
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1]

max_class == y_test:
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True False]

correct:
94.73684210526315



> 5 points

##### b. Print the **confusion matrix** for the test data of the 3 classes found by your own implementation of the logistic regressor. You can use `Sklearn`’s library; here’s an example of how:
<image style="width:50%" src="./1b-ex.png"/>

In [10]:
from sklearn.metrics import confusion_matrix

print(f"confusion_matrix:\n{confusion_matrix(y_test, max_class)}\n")

confusion_matrix:
[[13  0  0]
 [ 0 14  2]
 [ 0  0  9]]



> 5 points

##### c. Print the confusion matrix for the test data found by `sklearn`’s logistic regressor.

In [11]:
from sklearn.linear_model import LogisticRegression
logistic_regressor = LogisticRegression(max_iter=500)


In [12]:
fit_class = logistic_regressor.fit(X_train, y_train)
y_pred = fit_class.predict(X_test)
print(f"y_pred:\n{y_pred}\n")
print(f"y_test:\n{y_test}\n")
print(f"y_pred == y_test:\n{y_pred == y_test}\n")
print(f"correct:\n{np.sum(y_pred == y_test) / len(y_test) * 100}\n")
print(f"confusion_matrix:\n{confusion_matrix(y_test, y_pred)}\n")

y_pred:
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]

y_test:
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1]

y_pred == y_test:
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False]

correct:
97.36842105263158

confusion_matrix:
[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]



> 10 points

##### d. For each **mis-classified** data sample, show the classification probability for class 0, 1, and 2 
*we want to see that the probabilities are hovering around the 0.2 to 0.9 range for these data points*


In [13]:
y_test_prob = fit_class.predict_proba(X_test)
print("Prediction probability for test data P[0], P[1], P[2]: \n", y_test_prob)


Prediction probability for test data P[0], P[1], P[2]: 
 [[1.16879713e-04 5.59173946e-02 9.43965726e-01]
 [1.26309415e-02 9.60290623e-01 2.70784360e-02]
 [9.84388468e-01 1.56114930e-02 3.88752756e-08]
 [1.25811004e-06 2.34527966e-02 9.76545945e-01]
 [9.70274628e-01 2.97252070e-02 1.64616041e-07]
 [2.00954229e-06 5.97993123e-03 9.94018059e-01]
 [9.81934830e-01 1.80650983e-02 7.13675998e-08]
 [2.83358621e-03 7.47763906e-01 2.49402507e-01]
 [1.50589964e-03 7.39154135e-01 2.59339965e-01]
 [2.04941217e-02 9.35763785e-01 4.37420934e-02]
 [9.19538183e-05 1.60144101e-01 8.39763946e-01]
 [6.95834502e-03 8.10288425e-01 1.82753230e-01]
 [4.06402244e-03 7.93855231e-01 2.02080746e-01]
 [3.04877992e-03 7.60967799e-01 2.35983421e-01]
 [3.85935596e-03 7.10467056e-01 2.85673588e-01]
 [9.82822812e-01 1.71771312e-02 5.72475965e-08]
 [6.69996306e-03 7.56228367e-01 2.37071670e-01]
 [1.13766781e-02 8.44682142e-01 1.43941180e-01]
 [9.67687107e-01 3.23126753e-02 2.17465690e-07]
 [9.82895242e-01 1.71046979e-02

In [14]:
d = 2
# print with d decimal places and suppress scientific notation
np.set_printoptions(precision=d, suppress=True)
print(
    "Prediction probability for 5 rows of data P[0], P[1], P[2]: \n", y_test_prob)


Prediction probability for 5 rows of data P[0], P[1], P[2]: 
 [[0.   0.06 0.94]
 [0.01 0.96 0.03]
 [0.98 0.02 0.  ]
 [0.   0.02 0.98]
 [0.97 0.03 0.  ]
 [0.   0.01 0.99]
 [0.98 0.02 0.  ]
 [0.   0.75 0.25]
 [0.   0.74 0.26]
 [0.02 0.94 0.04]
 [0.   0.16 0.84]
 [0.01 0.81 0.18]
 [0.   0.79 0.2 ]
 [0.   0.76 0.24]
 [0.   0.71 0.29]
 [0.98 0.02 0.  ]
 [0.01 0.76 0.24]
 [0.01 0.84 0.14]
 [0.97 0.03 0.  ]
 [0.98 0.02 0.  ]
 [0.   0.19 0.81]
 [0.01 0.71 0.28]
 [0.94 0.06 0.  ]
 [0.98 0.02 0.  ]
 [0.   0.43 0.57]
 [0.99 0.01 0.  ]
 [0.95 0.05 0.  ]
 [0.01 0.9  0.09]
 [0.14 0.85 0.01]
 [0.96 0.04 0.  ]
 [0.   0.12 0.88]
 [0.01 0.68 0.3 ]
 [0.97 0.03 0.  ]
 [0.   0.36 0.64]
 [0.   0.03 0.97]
 [0.05 0.88 0.07]
 [0.94 0.06 0.  ]
 [0.   0.31 0.69]]
